# Learn Docker and Kubernetes faster

# 对比学习docker和kubernets
##  1.定位与作用  

| 方面 | Docker |kubernetes |
|--------|--------|---------|
|核心目标| 解决打包和运行应用的问题 | 解决大规模管理容器集群的问题 |
|适用范围| 单机或少量容器 | 大规模集群，多节点容器管理 |

kubernetes（以下用k8s代替）就像蜂后，掌管整个蜂巢，容器则是管理单个或少数个蜂窝

## 2.核心功能  
| 功能 | Docker | Kubernetes |
|--------|--------|---------|
|容器运行| 创建、启动、停止、删除容器| 不直接运行容器，依赖容器运行时（docker、containerd） |
|资源调度| 单机上由用户手动启动容器 | 自动调度容器到合适的节点（根据cpu，内存，负载等|
|打包镜像| docker build 将代码打包成镜像| 不提供打包功能，使用现成镜像|
|扩缩容| 需要手动运行多个容器并配置负载均衡| kubectl scale自动完成扩缩容|
|自愈能力| 容器挂了需人工重启 | pod挂了会自动重建，保证高可用|
|服务发现与网络| 默认容器之间用bridge网络（需自设），跨机困难| 内置service和ingress，支持跨主机的服务发现和负载均衡|
|存储管理| 支持挂载卷（Volumes)存储数据至本地| 提供持久化存储（PersistenVolume，PVC）|  

docker和k8s是相辅相成的

## 3.使用场景
|场景|推荐工具|
|----|-------|
你想快速打包并运行一个应用 | Docker|
你有一个小型服务，自己用，单机运行即可 | Docker|
你有几十/上百个容器，需要自动伸缩、负载均衡 | k8s|
你需要保证7x24小时高可用 | k8s |  

对于大需求来说，k8s是最佳选择，对于小需求，则docker更胜一筹
但k8s需要docker

# 学习Docker

## 入门
### 镜像
- 镜像是一个包含应用及其依赖的轻量级、独立、可执行的软件包，就像一张操作系统+软件的快照
- 镜像是静态的，不能运行

### 容器
- 容器是镜像的一个运行实例，是一个轻量级、可移植、自给自足的软件运行环境，相当于正在运行的程序
-是基于镜像启动的运行环境


### 容器与镜像实践
让我们实践一下，请在自行安装了docker后，在终端（Linux/Mac/WSL2/Docker Desktop)输入以下命令    

-docker --version 
>查看docker版本  

-docker pull ubuntu 
>拉取你的第一个官方镜像

-docker run -it ubuntu bash  
 >运行一个ubuntu容器，进入交互模式（-i -t通常合起来使用，前者提供输入渠道，后者提供命令行交互界面，bash 是容器启动后执行的第一个进程，即打开一个 Bash Shell。）

-docker ps 
>查看正在运行的容器  

-docker ps -a 
>查看历史上运行过的所有容器  

-exit 
>退出容器

### 网络
|模式 | 命令 | 特点| 
|--------|--------|---------|
|bridge(default)| docker run -it --name c1 ubuntu| 容器有独立ip，通过Docker的虚拟网桥和主机通信|
|host| docker run --network host nginx | 容器直接使用宿主机网络，没有隔离|
|none| docker run --network none ubuntu| 容器无网络|

### 网络小实验
```bash
#启动两个容器
docker run -it --name c1 ubuntu bash 
docker run -it --name c2 ubuntu bash
#在c1内安装ping工具后，尝试ping c2
apt update && apt install -y iputils-ping
ping c2
```
结果是什么呢？

因为默认是bridge模式，容器间无法通过名称通信，可以通过docker inspect c2查看c2的ip地址，然后ping这个ip即可
```bash
#查看c2的ip地址
docker inspect c2 | grep IPAddress

#ping这个ip地址
ping <ip地址>
```
还可以怎么做呢？
```bash
#让两个容器加入同一个自定义网络
docker network create mynet
docker run -it c1 --name mynet ubuntu bash
docker run -it c2 --name mynet ubuntu bash

#在c1容器里
ping c2
```
恭喜你，网络大师！

### 数据卷（Volumes）
容器内的数据默认会随容器销毁而丢失，要保存数据，我们需要使用volume  

让我们来做一个小实验
```bash
#创建一个挂载卷的容器
docker run -it -v /mydata:/data ubuntu bash

#在容器里写文件
echo "hello docker" > /data/test.txt

#回到宿主机，查看文件是否存在
cat /mydata/test.txt 

#删除容器
docker rm ubuntu

#再次查看文件
cat /mydata/test.txt 
```
我们在创建容器时，顺带创建了一个文件夹（挂载卷），这样，即使删除容器后，挂载卷内的文件，依旧会保存在宿主机中

**恭喜你,你成功入门了docker，接下来，我们将学习更进一步的知识，easy，你已经很强啦！我和AI都会帮助你的！**

## 进阶
### Dockerfile！！！
Dockerfile是一个文本文件，包含了一系列指令，用于定义如何构建一个Docker镜像。通过编写Dockerfile，可以自动化地创建和配置镜像，使得应用的部署更加高效和一致。  
换而言之，Dockerfile就像是一份“造镜像的食谱”  
* 写好之后->用docker build->得到镜像->再用docker run启动容器  
  
是不是感觉太高深啦？哈哈哈哈，我们会从简单实用的蒸水煮蛋开始，谁说它不美味呢？谁又说会蒸水煮蛋就不是一个会做饭的人呢？


#### 水煮蛋实战！

```bash
mkdir hello-docker && cd hello-docker 
#在当前目录下，创建一个名为hello-docker的文件夹，并进入这个文件夹

vim dockerfile
#使用vim创建一个dockerfile文件

#进入之后，摁i进入编辑模式，写入以下代码
FROM alpine:lastest
#使用最小的基础镜像
CMD ["echo","hello from Docker!"]
#容器启动时执行的命令
#现在摁：esc、wq、enter键回到bash中

docker build -t hello-docker .
#创建镜像，-t指定docker名称,最后的. 表示从当前目录读取dockerfile文件来构建镜像

docker run hello-docker
#运行容器
```
猜一猜最后会输出什么呢？
如果我把 CMD ["echo", "Hello from Docker!"] 换成 CMD ["echo", "你好，世界！"]，输出会变吗？

镜像是静态的“食谱”，那我能不能用同一个镜像跑出 3 个容器？
（提示：像一份菜谱能做多盘菜）

#### 进阶水煮蛋实战！
这次我们挑战写一个带python程序的dockerfile
```bash
mkdir python-docker && cd python-docker

vim app.py #创建app.py文件并进入
#摁i进入编辑模式写入以下代码
print("Helo from Python in Docker!")
#这段代码会输出""里的内容
#我们退出vim，进入bash

vim dockerfile #创建dockerfile
#写入以下代码
FROM python:3.9-slim #使用官方python镜像作为基础镜像

COPY app.py /app/app.py #将当前目录的app.py拷入容器内

WORKDIR /app #设置工作目录

CMD ["python","app.py"] #启动容器时运行python脚本

#退出vim进入bash

doker build -t python-docker

docker run python-docker
```
这次会输出什么呢？
如果你把 CMD ["python", "app.py"] 换成 ENTRYPOINT ["python", "app.py"]，会有什么区别？

试着修改 app.py，让它输出你自己的名字（比如 Hello, I'm XXX from Docker!），然后重新构建并运行镜像。

### Docker Compose!!
当我们需要一次性启动多个容器的时候，用终端一个个敲实在是太麻烦啦！  
这时候懒人工具docker compose就出现了  
Docker Compose可以用docker-compose.yml文件，将多个容器的配置写好，然后一件启动  
让我们进入实战！  
目标：写一个简单的Web服务+Redis数据库的Compose项目
```bash
mkdir compose-demo && cd compose-demo

vim app.py

#输入以下代码
from flask import Flask
import redis

app=Flask(__name__) #创建flask应用实例
r=redis.Redis(host="redis",port=6379)

@app.route("/") #装饰器，将hello函数与根路径关联。当用户访问根路径，flask会调用hello函数
def hello():
    count=r.incr("hits") #每次访问+1
    return f"Hello from Flask!You are visitor #{count}"

if __name__ =="__main__":
    app.run(host="0.0.0.0",port=5000)

#退出vim进入bash
vim requirements.txt
#写入
flask
redis
#退出vim

vim dockerfile
#写入
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt
RUN pip install -r requirements.txt
COPY app.py .
CMD ["python","app.py"]
#退出vim

vim docker-compose.yml
#写入
version:"3"
services: #表示要运行的服务，对应一个容器或一组容器
    web:     
     build:. #根据当前目录的dockerfile构建镜像
     ports:
       -"5000：5000" #将宿主机的端口映射到容器里，左边是本机访问端口，右边是容器里应用的端口
     depends_on: #表示服务的依赖关系，这里是web依赖于redis
       -redis #先启动redis再启动web，但不保证redis已经准备好

    redis:
      image:"redis:alpine" #redis的镜像

#退出vim
docker compose up #启动项目
```
在本地浏览器访问http://localhost:5000

你应该会看到:Hello from Flask! You are visitor #1
尝试刷新页面，哪里会发生变化呢？


实战加强
我们针对上一个实验，让他变得更强。  
之前提到过，容器在删除后会丢失所有数据，但是我们可以使用volumes来持久化数据或共享文件  
其次，容器内的应用可能不是真正可用的，我们需要采取措施进行检查  
最后，为了确保我们的程序安全，通常会将密码、配置写放在.env文件中。  
接下来我们将针对这三点对上面的程序进行改良

```bash
cd compose-demo
vim docker-compose.yml
#在redis部分加入volume持久化和healthcheck（健康检查）
redis：
  image:"redis:alpine"
  volumes：
    - ./redis-data:/data
  healthcheck:
    test:["CMD","redis-cli","ping"]
    intervaal:5s #每5秒执行一次redis-cli ping
    timeout:3s   #超过3秒没响应算失败
    retries:5    #连续5次失败判定容器unhealthy
#退出vim
```
这样一来，就能够记录下click的次数而不用担心容器删除导致的数据丢失，同时也可以让web服务等到redis真的做好准备后再去工作  
接下来，我们要解决安全的问题
```bash
vim .env
#写入
REDIS_HOST=redis
REDIS_PORT=6378
#退出vim
vim docker-compose.yml
#找到web
web:
  build: .
  prots:
    - "5000:5000"
  environment:
    - REDIS_HOST=${REDIS_HOST}
    - REDIS_RORT=${REDIS_PORT}
    #这样一来，后续我们就可以直接在.env上切换配置了
#退出vim

docker compose down #停止并删除容器
docker compose up #再次启动容器

docker ps #列出当前正在运行的docker容器
docker inspect <redis_container_id> | grep Health -A 5
#这行代码中的<>请替换为ps查到的demo容器id，我们通过它，可以看到容器的健康状态
```
到这里，我们就掌握了基本的docker操作，我们还创建了一个以redis为数据库的web程序，怎么样？厉不厉害？
